# Darcy equation

In this tutorial we present how to solve a Darcy equation with [PyGeoN](https://github.com/compgeo-mox/pygeon).  The unkwons are the velocity $q$ and the pressure $p$.

Let $\Omega=(0,1)^2$ with boundary $\partial \Omega$ and outward unit normal ${\nu}$. Given 
$k$ the matrix permeability, we want to solve the following problem: find $({q}, p)$ such that
$$
\left\{
\begin{array}{ll}
\begin{array}{l} 
k^{-1} {q} + \nabla p = {0}\\
\nabla \cdot {q} = 0
\end{array}
&\text{in } \Omega
\end{array}
\right.
$$
with boundary conditions:
$$ p = 0 \text{ on } \partial_{top} \Omega \qquad p = 1 \text{ on } \partial_{bottom} \Omega \qquad \nu \cdot q = 0 \text{ on } \partial_{left} \Omega \cup \partial_{right} \Omega$$

We present *step-by-step* how to create the grid, declare the problem data, and finally solve the problem.

First we import some of the standard modules, like `numpy` and `scipy.sparse`. Since PyGeoN is based on [PorePy](https://github.com/pmgbergen/porepy) we import both modules.

In [1]:
import numpy as np
import scipy.sparse as sps

import porepy as pp
import pygeon as pg

We create now the grid, since we will use a Raviart-Thomas approximation for ${q}$ we are restricted to simplices. In this example we consider a bi-dimensional structured grid, but the presented code will work also in 1d and 3d. PyGeoN works with mixed-dimensional grids, so we need to convert the grid.

In [2]:
N = 10
sd = pp.StructuredTriangleGrid([N] * 2, [1] * 2)
# convert the grid into a mixed-dimensional grid
mdg = pp.meshing.subdomains_to_mdg([sd])

With the following code we set the data, in particular the permeability tensor and the boundary conditions. Since we need to identify each side of $\partial \Omega$ we need few steps.

In [3]:
key = "flow"
bc_val = []
bc_ess = []
for sd, data in mdg.subdomains(return_data=True):
    # permeability tensor
    perm = pp.SecondOrderTensor(np.ones(sd.num_cells))
    
    # with the following steps we identify the portions of the boundary
    # to impose the boundary conditions
    b_faces = sd.tags["domain_boundary_faces"].nonzero()[0]

    b_face_centers = sd.face_centers[:, b_faces]
    out_flow = b_face_centers[1, :] == 1
    in_flow = b_face_centers[1, :] == 0
    no_flow = np.logical_or.reduce((b_face_centers[0, :] == 1, b_face_centers[0, :] == 0))

    labels = np.array(["neu"] * b_faces.size)
    labels[np.logical_or(in_flow, out_flow)] = "dir"

    bc = pp.BoundaryCondition(sd, b_faces, labels)

    faces, _, sign = sps.find(sd.cell_faces)
    sign = sign[np.unique(faces, return_index=True)[1]]
        
    bc_val.append(np.zeros(sd.num_faces))
    bc_val[-1][b_faces[in_flow]] = sign[b_faces[in_flow]] * np.ones(b_faces[in_flow].size)

    bc_ess.append(np.zeros(sd.num_faces + sd.num_cells, dtype=bool))
    bc_ess[-1][b_faces[no_flow]] = True

    parameters = {
        "second_order_tensor": perm,
        "bc": bc,
    }
    pp.initialize_data(sd, data, key, parameters)

Once the data are assigned to the mixed-dimensional grid, we construct the matrices. In particular, the linear system associated with the equation is given as
$$
\left(
\begin{array}{cc} 
M & B^\top\\
B & 0
\end{array}
\right)
\left(
\begin{array}{c} 
q\\ 
p
\end{array}
\right)
=\left(
\begin{array}{c} 
p_{\partial}\\ 
0
\end{array}
\right)
$$<br>
where $p_{\partial}$ is the vector associated to the pressure boundary contions. To construct the saddle-point problem, we rely on the `scipy.sparse` function `bmat`. Once the matrix is created, we also construct the right-hand side containing the boundary conditions.

In [4]:
# construct the local matrices
mass = pg.face_mass(mdg)
div = pg.div(mdg)

# assemble the saddle point problem
spp = sps.bmat([[mass, div.T], 
                [ div,  None]], format="csc")

# get the degrees of freedom for each variable
dof_p, dof_q = div.shape

# assemble the right-hand side
rhs = np.zeros(dof_p + dof_q)
rhs[:dof_q] += np.hstack(bc_val)

We need to solve the linear system, PyGeoN provides a framework for that. The actual imposition of essential boundary conditions (flux boundary conditions) might change the symmetry of the global system, the class `pg.LinearSystem` preserves this structure by internally eliminating these degrees of freedom. Once the problem is solved, we extract the two solutions $q$ and $p$.

In [5]:
# solve the problem
ls = pg.LinearSystem(spp, rhs)
ls.flag_ess_bc(np.hstack(bc_ess), np.zeros(dof_q + dof_p))
x = ls.solve()

# extract the variables
q = x[:dof_q]
p = x[-dof_p:]

Since the computed $q$ is one value per facet of the grid, for visualization purposes we project the flux in each cell center as vector. We finally export the solution to be visualized by [ParaView](https://www.paraview.org/).

In [8]:
# post process velocity
proj_q = pg.proj_faces_to_cells(mdg)
cell_q = (proj_q * q).reshape((3, -1), order="F")

for _, data in mdg.subdomains(return_data=True):
    data[pp.STATE] = {"cell_q": cell_q, "p": p}

save = pp.Exporter(mdg, "sol")
save.write_vtu(["cell_q", "p"])

A representation of the computed solution is given below, where the cells are colored with $p$ and the arrows are the $q$. <br>
![](fig/darcy.png)